# Pre-class Reading: Day 17 (Nov 06, 2023)<br>Monte Carlo Methods 2
Learning goals
1. Apply Monte Carlo error propagation methods to determine the error on a calculated quantity
1. Apply Markov Chain Monte Carlo methods to simple systems

## *No self-assessment questions for this reading*

Similar to the previous reading assignment, the ideas presented here will mostly be new so please engage with the entire reading assignment.

## 17.1 Intro to Monte Carlo Error Propagation

Monte Carlo error propagation starts with the notion that one representation of a measurement uncertainty (aka measurement error) is that of a gaussian where the mean of the gaussian represents the measurand (the value) and the standard deviation represents the measurement error. 

For example, to determine the resistance ($R$) of a certain portion of my very noisy circuit, I can measure the voltage drop ($V$) across that resistance and the current ($I$) through that resistance. In my example, let's say I measured the following values:
* $V=(1.52 \pm 0.11)\mbox{ V}$, and
* $I=(0.0642 \pm 0.0029)\mbox{ A}$.

As a reminder, the calculus method for error propagation for a function ($f$) determined from uncorrelated variables ($x_1, x_2, ...$), the error $df$ is given by:

$$f = f(x_1, x_2, ..),$$

$$df = \sqrt{\sum^N_{i=1} (dx_i\frac{\partial f}{\partial x_i})^2}.$$

To find the erorr in the resistance, this would look like:

$$ R = \frac{V}{I},$$

$$ dR = \sqrt{(dV\frac{\partial R}{\partial V})^2 + (dI\frac{\partial R}{\partial I})^2},$$

$$ dR = \sqrt{(dV \cdot \frac{1}{I})^2 + (dI\frac{V}{I^2})^2},$$

$$ dR = R \sqrt{(\frac{dV}{V})^2 + (\frac{dI}{I})^2},$$

where the final step requires a little algegra and a substitution of $R=V/I$ back in. 

If you work through the arithmetic using the numbers give, you should get

$$R = (23.7 \pm 2.0)\, \Omega.$$

Again, with the notion that the error in a measurement can be represented as the standard deviation of a Gaussian, we could represent each of these values as follows:

In [ ]:
# Run me to generate graphical representations of these measurements

import numpy as np
import matplotlib.pyplot as plt

# Define the Gaussian function
def gaussian(x, mu, sigma):
    return (1 / (sigma * np.sqrt(2 * np.pi))) * np.exp(- 0.5 * ((x - mu) / sigma)**2)

# Given and calculated measurements
V, dV = 1.52, 0.11 # V
I, dI = 0.0642, 0.0029 # A
R, dR = 23.7, 2.0 # ohms

# Generate x and Gaussian y values
x_V = np.linspace(0, V + 4*dV, 1000)
x_I = np.linspace(0, I + 4*dI, 1000)
x_R = np.linspace(0, R + 4*dR, 1000)
y_V = gaussian(x_V, V, dV)
y_I = gaussian(x_I, I, dI)
y_R = gaussian(x_R, R, dR)

# Make our plots
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(8, 9))

axes[0].plot(x_V, y_V)
axes[0].fill_between(x_V, y_V, alpha=0.4)
axes[0].set_title(f"$V = ({V} \pm {dV})$ V")
axes[0].set_xlabel("Voltage (V)")

axes[1].plot(x_I, y_I)
axes[1].fill_between(x_I, y_I, alpha=0.4)
axes[1].set_title(f"$I = ({I} \pm {dI})$ A")
axes[1].set_xlabel("Current (A)")

axes[2].plot(x_R, y_R)
axes[2].fill_between(x_R, y_R, alpha=0.4)
axes[2].set_title(f"$R = ({R} \pm {dR})\,\Omega$ ")
axes[2].set_xlabel("Resistance ($\Omega$)")

for ax in axes:
    ax.set_yticklabels([])
    ax.grid(True)   

plt.tight_layout()
plt.show()


However, another approach to this error propagation would be to use Monte-Carlo approaches to generate a bunch of $V$ and $I$ data drawn from the above distribution and then for each pair of $V$ and $I$ points, calculate $R$. The mean and standard devation of this $R$ distribution should then be $R \pm dR$:

$$R_{\mbox{Monte Carlo}} = \frac{V_{\mbox{Monte Carlo}}}{I_{\mbox{Monte Carlo}}}.$$

Note that we are using only 5000 data points here so that the graphs below show our Monte Carlo data clearly. However, it would be reasonable to make `N` as high as one million (1,000,000) to make this quite accurate. 

In [ ]:
N = 5000

# Draw random samples from Gaussian distributions for V and R
V_samples = np.random.normal(V, dV, N)
I_samples = np.random.normal(I, dI, N)

# Calculate R for each pair of V and I samples
R_samples = V_samples / I_samples 

# Compute mean and standard deviation of the R samples
R_mc = np.mean(R_samples)
dR_mc = np.std(R_samples, ddof=1)

print(f"Calculated R = {R_mc:.1f} ± {dR_mc:.1f} ohms")

Let's plot these like we did above to see them instead

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Make our plots
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(8, 9))

# Plot Gaussian and histogram for V
axes[0].plot(x_V, y_V)
axes[0].fill_between(x_V, y_V, alpha=0.4)
axes[0].hist(V_samples, bins=50, density=True, alpha=0.8)
axes[0].set_title(f"$V = ({V} \pm {dV})$ V")
axes[0].set_xlabel("Voltage (V)")

# Plot Gaussian and histogram for I
axes[1].plot(x_I, y_I)
axes[1].fill_between(x_I, y_I, alpha=0.4)
axes[1].hist(I_samples, bins=50, density=True, alpha=0.8)
axes[1].set_title(f"$I = ({I} \pm {dI})$ A")
axes[1].set_xlabel("Current (A)")

# Plot Gaussian and histogram for R
axes[2].plot(x_R, y_R)
axes[2].fill_between(x_R, y_R, alpha=0.4)
axes[2].hist(R_samples, bins=50, density=True, alpha=0.8)
axes[2].set_title(f"$R(calculated) = ({R} \pm {dR})\,\Omega$;    $R(Monte Carlo) = ({R_mc:.1f} \pm {dR_mc:.1f})\,\Omega$ ")
axes[2].set_xlabel("Resistance ($\Omega$)")

for ax in axes:
    ax.set_yticklabels([])
    ax.grid(True)

plt.tight_layout()
plt.show()

### Your turn

Verify that Monte Carlo error propagation gives a consistent result $dz$ to that from the calculus method of error progagation for the following situation.

Population growth $z$ is predicted by a combination of available resources $x$ and a predator-prey relationship $y$ as given by the following expression:

$$ z = \ln x + y^2.$$

Using the calculus method for error derivation, we would find that the uncertainty in this population growth is given by

$$ dz = \sqrt{\left(\frac{dx}{x}\right)^2 + (2\,y\,dy)^2}.$$

For the following values, use Monte Carlo error propagation to show that you get the same $dz$ as we get from the direct calculation:

* $x = 2.03 \pm 0.15$,
* $y = 0.679 \pm 0.052$.

In [ ]:
# Code to find the directly calculated values

x_val, dx_val = 2.03, 0.15
y_val, dy_val = 0.679, 0.052

z_calc = np.log(x_val) + y_val**2
dz_calc = np.sqrt((dx_val/x_val)**2 + (2*dy_val*y_val)**2)

print(f"Calculated z = {z_calc:.2f} ± {dz_calc:.2f}")

In [ ]:
# Your code for Monte Carlo error propagation

N = 1000000



## 17.2 Introduction to the Monte Carlo Markov Chain Method 

Markov Chain Monte Carlo methods are algorithms where the the progression of the state depends only on the previous state vector and not the entire history of the state. In the following sections we will build up to a simple model of an ideal gas with quantized energy states by first starting out with modelling the random walk problem.

The general approach is as follows (it will make much more sense when we look at our examples):

1. **Initialization**: Start from an initial state.
2. **Proposal**: Propose a move to change to a new state based on the current one. The way this proposal happens is specified by a proposal distribution.
3. **Acceptance**: Decide whether to accept the move to the new state or stay with the current one.
4. **Iteration**: Repeat the proposal and acceptance steps many times.

## 17.3 Random walk for one particle

*The following two sections draw on examples from Ayars textbook: https://freecomputerbooks.com/Computational-Physics-with-Python-by-Eric-Ayars.html*

Brownian motion can model the motion of a particle, such as smoke, in a gas. Random collisions with the gas molecules cause the smoke to undergo a motion described as a random walk. We will look at a one-dimensional random walk, where at each iteration will be a step in either the positive or negative direction.

Based on our general approach introduced earlier, this would look as follows:

1. **Initialization**: Start from an initial state.
  * Start the particle at `x = 0`<br><br>
2. **Proposal**: Propose a move to change to a new state based on the current one. The way this proposal happens is specified by a proposal distribution.
  * Randomly determine if the particle will move in the positive or negative direction, with equal probability.<br><br>
3. **Acceptance**: Decide whether to accept the move to the new state or stay with the current one.
  * For this initial model, we will accept all proposed moves. But you could imagine an example of the smoke particle and gas being in a room such that proposals for the smoke to move through the wall would be rejected.<br><br>
4. **Iteration**: Repeat the proposal and acceptance steps many times.
  * We will iterate 100 times.<br><br>

In [ ]:
# Run me multiple times to see the different ways the system can evolve

N = 100  # number of steps

# Store locations after each step and times
x = [0]
t = range(N)

# The possible choices
dir_choices = [-1, 1]

# random walk
for i in range(1, N):
    step = np.random.choice(dir_choices)
    x_new = x[-1] + step # move one step in the pos/neg direction
    x.append(x_new)
    

plt.plot(t, x)
plt.xlabel('Time Steps')
plt.ylabel('Position')
plt.grid(True)
plt.show()

## 17.4 Random walk for a collection of particles

We now expand our random walk to 200 particles and look at how the overall behaviour of this system will evolve over time. As you have seen when you ran the single-particle code above multiple times, some particles will wander off far in the positive or negative directions and others will remain somewhat close to x = 0. You will no be surprised to learn that if we look at this behaviour for multiple particles at once, we will expect the positions of all particles at a specific time to follow a Gaussian distribution. Further to that, we can also expect that the standard deviation of this Gaussian distribution should evolve like the square root of the number of steps taken.

Summarizing our approach for a collection of particles:

1. **Initialization**: Start from an initial state.
  * Start with 2000 particles at `x = 0`<br><br>
2. **Proposal**: Propose a move to change to a new state based on the current one. The way this proposal happens is specified by a proposal distribution.
  * Randomly determine if *each* particle will move in the positive or negative direction, with equal probability.<br><br>
3. **Acceptance**: Decide whether to accept the move to the new state or stay with the current one.
  * Accept all proposed moves (no bounding walls)<br><br>
4. **Iteration**: Repeat the proposal and acceptance steps many times.
  * We will iterate 200 times.<br><br>
  
Work through the code below carefully to make sense of each step. We use two visualizations to summarize the behaviour of the system. The first is a histogram of all 2000 particles in the system after the final step. Notice that the mean of this is approximately zero. Second is a history of now the mean and standard deviation of `x` for all particles evolved over time. Here we see that the mean remains approximately zero and the standard deviation evolves like the square root of the number of time steps. 

In [ ]:
from scipy import optimize

nsteps = 200 # number of steps
nparticles = 2000  # number of particles

# Initial positions of all particles
x = np.zeros(nparticles)

# Times
t = range(nsteps)

# Arrays to store average and standard deviations of x of all particles at each time
x_mean = np.zeros(nsteps)
x_std = np.zeros(nsteps)

# Evolve the system through each time step
for i in range(1,nsteps):
    
    # Propose the next move for all particles at the same time
    step = np.random.choice([-1,1], size=nparticles)
    
    # Accept all proposed moves
    # - Each particle in x has its own random step
    x = x + step
    
    # Characterize the new state of the system
    x_mean[i] = np.mean(x)
    x_std[i] = np.std(x, ddof=1)

# Fit the evolution of x_std to a*std(t)  

# First define the fitting function
def fun (x, a):
    return a * np.sqrt(x)

# And then perform the fit
popt, pcov = optimize.curve_fit(fun, t, x_std)    
    
# Plotting
fig, axs = plt.subplots(2, 1, figsize=(10, 8))

# Histogram of final positions
axs[0].hist(x, bins=20)
axs[0].set_title(f'Histogram of Particle Positions After {nsteps} Steps')
axs[0].set_xlabel('Position')
axs[0].set_ylabel('Number of Particles')

# Plot of mean and standard deviation over time
axs[1].plot(t, x_mean, label='Mean Position', linestyle=':')
axs[1].plot(t, x_std, label='Position STD')
axs[1].plot(t, fun(t,*popt), label='fit: sqrt(t)', linestyle='--')
axs[1].set_title('Mean and Standard Deviation of Particle Positions Over Time')
axs[1].set_xlabel('Time step')
axs[1].set_ylabel('Position')
axs[1].legend()

plt.tight_layout() 
plt.show()

### 17.5 A quantum ideal gas with one atom

*The following examples are based on examples from "Computational Physics" (revised edition) by Mark Newman.*

In this example, we will be discussing energies in quantum systems. If this is a completely new idea to you, the main idea that you need to make sense of is that our atom has discrete energy levels and we are going to do a random walk up and down between these energy levels. When a move is proposed to decrease the atom's energy from an excited state ($n>1$), it will always be accepted (systems tend toward states of lower energy). However, a move that proposes to increase the atom's energy will only be accepted sometimes, where the probability that it is accepted decreases the larger that this proposed change in energy would be with respect to the thermal energy of the system.

We are going to model an ideal gas as a collection of non-interacting atoms inside a box of length L. Like you have encountered with the Bohr atom, the energy of such an atom goes like $n^2$, where $n$ is the principal quantum number, which can only take on values of 1, 2, 3, ...

Formally, the energy of each atom, of mass $m$ is

$$E(n) = \frac{\pi^2 \hbar^2}{2 m L}n^2.$$

We will choose a system of units such that we can say $m = L = \hbar = 1$, such that the above equation reduces to

$$E(n) = \frac{\pi^2}{2}n^2.$$

We will first look at a one-atom "ideal gas", which is mostly nonsensical, but will help us understand the behaviour of a single atom once we make it a multi-atom gas. Our system has a total thermal energy of $k_B T$ (the Boltzmann constant times temperature gives a thermal energy) and our atom has an energy that can randomly increase or decrease many times. 

An increase in energy corresponding to $n=+1$ would have the factor $n^2$ increase by

$$\Delta(n^2) = (n+1)^2 - n^2 = 2n  + 1,$$ 

and thus the total energy would change by

$$dE(\Delta n = +1) = \frac{\pi^2}{2}(2n+1).$$

Similarly, a decrease in energy corresponding to $n=-1$ would be

$$dE(\Delta n = -1) = \frac{\pi^2}{2}(-2n+1).$$

Finally, we introduce the Metropolis probability which tells us how likely it is that a proposed move that would change the energy of the system by an amount $dE$ will be accepted. We won't concern ourselves with how this was derived for now. As discussed above, when the atom is in an excited state and the proposed change in energy would decrease the energy of the system, it is always accepted. When the proposed change in energy would increase the energy of the system, it is accepted with the probability

$$p = e^{- k_B T (dE)}.$$

Summarizing our approach for a one atom ideal gas:

1. **Initialization**: Start from an initial state.
  * Start with an atom in an initial quantum state of $n=1$.<br><br>
2. **Proposal**: Propose a move to change to a new state based on the current one. The way this proposal happens is specified by a proposal distribution.
  * Randomly determine if the atom will try to increase or decrease its energy. These moves are proposed with equal probability, even if they are not accepted with equal probability<br><br>
3. **Acceptance**: Decide whether to accept the move to the new state or stay with the current one.
  * Accept all moves that would <u>decrease</u> the energy of the system from an excited state $n>1$. Reject a move that would try to <u>decrease</u> the energy of the ground state ($n=1$). Accept a move that would <u>increase</u> the energy of the system according to the Metropolis probability. <br><br>
4. **Iteration**: Repeat the proposal and acceptance steps many times.
  * Iterate 200 times.<br><br>
  
Run the code below and observe how the particle jumps out of the ground state into the first excited state and sometimes even higher excited states over time, but continues to decay back to the ground state. Try playing around with the total thermal energy of the system `Temp` to see how this impacts the overall behaviour. 

In [ ]:
# The one atom ideal gas

Temp = 20 # k_B*T
Nsteps = 200 # Number of steps

# Store the initial quantum state
n = 1

# Initial energy of our gas molecule:
#   E(n) = n^2 * pi^2 * hbar^2 / (2 * m * L^2), 
#   where hbar = m = L = 1
E = n**2 * np.pi**2 / 2

# Array to store the energy evolution over time
E_t = np.zeros(Nsteps)

# Our main loop
for j in range(Nsteps):

    # Choose the proposed move
    step = np.random.choice(["up", "down"])
    
    if step == "up":
        dn = 1
        dE = (2*n + 1) * np.pi**2 / 2
    else:
        dn = -1
        dE = (-2*n + 1) * np.pi**2 / 2

    # Decide whether to accept the move
    # - We reject the downward move (dn = -1) if the molecule is already in the 
    #   the ground state n = 1.
    # - When dE is negative ("down"), -dE/T will be positive
    #   and thus np.exp(-dE/T) will be > 1 and the move will 
    #   always be accepted
    # - When dE is positive ("up"), -dE/T will negative
    #   and there is a finite probability we will accept the move
    #   given by np.exp(-dE/T)
    if not (n == 1 and dn == -1): 
        if np.random.random() < np.exp(-dE/Temp):
            n += dn
            E += dE
    
    E_t[j] = E

# Make the graph
plt.plot(E_t)
plt.xlabel("Time")
plt.ylabel("Energy")
plt.show()

### 17.6 A quantum ideal gas with many atoms

We extend the previous example to a system with multiple particles (`Natoms`) and modify our algorithm so that in each step we propose only a single move by first picking one of the atoms at random and then proposing that it move up or down in energy. 

Notice that the system seems to reach some sort of equilibrium for the total energy of the ideal gas (all of the atoms) within approximately 10,000 steps. 

Again, try playing around with the temperature, but also the number of atoms. How does varying these parameters impact the equilibrium energy and how long it takes to get to that equilibrium?

In [ ]:
# The multi-atom ideal gas

Temp = 20 # k_B*T
Nsteps = 50000 # Number of steps
Natoms = 1000

# Store the initial quantum states of all atoms
n = np.ones(Natoms)

# Initial energy of our our system
#   E(n) = n^2 * pi^2 * hbar^2 / (2 * m * L^2), 
#   where hbar = m = L = 1
E = Natoms * np.pi**2 / 2

# Array to store the energy evolution over time
E_t = np.zeros(Nsteps)

# Our main loop
for j in range(Nsteps):

    # Choose which atom will have the proposed move
    atom = np.random.randint(Natoms)
    
    # Choose the proposed move
    step = np.random.choice(["up", "down"])
    
    if step == "up":
        dn = 1
        dE = (2*n[atom] + 1) * np.pi**2 / 2
    else:
        dn = -1
        dE = (-2*n[atom] + 1) * np.pi**2 / 2

    # Decide whether to accept the move
    # - We reject the downward move (dn = -1) if the molecule is already in the 
    #   the ground state n = 1.
    # - When dE is negative ("down"), -dE/T will be positive
    #   and thus np.exp(-dE/T) will be > 1 and the move will 
    #   always be accepted
    # - When dE is positive ("up"), -dE/T will negative
    #   and there is a finite probability we will accept the move
    #   given by np.exp(-dE/T)
    if not (n[atom] == 1 and dn == -1): 
        if np.random.random() < np.exp(-dE/Temp):
            n[atom] += dn
            E += dE
    
    E_t[j] = E

# Make the graph
plt.plot(E_t)
plt.xlabel("Time")
plt.ylabel("Total energy of the ideal gas")
plt.show()

## *Submitting this reading assignment*
Before submitting your work, please ensure you have worked carefully through all the cells. Afterward choose: File >> Save_and_Export_Notebook_As >> HTML. This will download an HTML version of your notebook to your computer which you can upload to Canvas